# Gradient-based sampling for PDE Bayesian Problems

Here we build a Bayesian problem to infer the conductivity in a Poisson problem (applications includes EIT problems).

## Learning objectives of this notebook:
1. Build a FEniCS based Poisson problem 
2. Build and solve the Bayesian problem in CUQIpy
	- Use Matern covariance operator for the prior
	- Use pCN sampler
3. Use gradient-based sampler
	- The chain rule
	- Use NUTS sampler 
4. Explore the effect of changing observation operator
	- Observe on the boundaries

## Import required libraries and set up configuration

In [ ]:
from scipy import optimize
import ufl
import matplotlib.pyplot as plt
import sys
#sys.path.append("../")
import cuqi
import cuqipy_fenics
import dolfin as dl
import numpy as np
cuqi.__version__

## 1. Build a FEniCS based Poisson problem 

The PDE model we consider here is a 2D steady-state problem (Poisson):

$$ \nabla \cdot \left(\kappa(\mathbf{x}) \nabla u(\mathbf{x})\right) = f(x) \;\;\;\;\mathrm{ in }\; (0,1)\times(0,1),$$

$$u(0,y)=0 \;\;\;\;\mathrm{on}\; x=0$$ 
$$u(1,y)=0 \;\;\;\;\mathrm{on}\; x=1$$ 
$$\kappa(x)\nabla u(x,0)\cdot n=0 \;\;\;\;\mathrm{on}\; y=0$$ 
$$\kappa(x)\nabla u(x,1)\cdot n =0 \;\;\;\;\mathrm{on}\; y=1$$ 



- where $\kappa(x)$ is the conductivity, $u(x)$ is the PDE solution (potential), $f(x)$ is the source term.

- We use the parameterization $\kappa(x) = e^{m(x)}$, to ensure positivity.

### The discretization 

We use finite element discretization of the model above where the solution and the parameters are approximated in a second and first order Lagrange polynomial space, respectively.  

We denote the discretized system that we need to solve as
$\mathbf{A}(\mathbf{m})\mathbf{U} = \mathbf{F}$
- $\mathbf{A}$ is the discretized diffusion differential operator
- $\mathbf{m}$ is the discretized Bayesian parameter 
- $\mathbf{U}$ is the discretized solution (the potential)
- $\mathbf{F}$ is the discretized RHS (the source term) 

### 1.1. Set up mesh

In [ ]:
nx = 12 # number of vertices on the x dimension
ny = 12 # number of vertices on the y dimension
mesh = dl.UnitSquareMesh(nx, ny) # create FEniCS mesh
dl.plot(mesh)

### 1.2. Set up function spaces

In [ ]:
solution_function_space = dl.FunctionSpace(mesh, 'Lagrange', 2) # function space for solution u
parameter_function_space = dl.FunctionSpace(mesh, 'Lagrange', 1) # function space for parameter m

### 1.3. Set up Dirichlet boundaries

In [ ]:
# Function (where do we have Dirichlet BC)
def u_boundary(x, on_boundary):
    return on_boundary and ( x[0] < dl.DOLFIN_EPS or x[0] > 1.0 - dl.DOLFIN_EPS)

# Expression (what is the value on these Dirichlet BC)
dirichlet_bc_expr = dl.Expression("0", degree=1)

# FEniCS Dirichlet BC Object
dirichlet_bc = dl.DirichletBC(solution_function_space,
                              dirichlet_bc_expr,
                              u_boundary) 

### 1.4. Set up source term

In [ ]:
f = dl.Constant(1.0)

### 1.5. Set up PDE variational form


In [ ]:
def form(m,u,p):
    return ufl.exp(m)*ufl.inner(ufl.grad(u), ufl.grad(p))*ufl.dx - f*p*ufl.dx

### 1.6 Create **CUQI** PDE object


In [ ]:
PDE = cuqipy_fenics.pde.SteadyStateLinearFEniCSPDE( 
        form,
        mesh, 
        parameter_function_space=parameter_function_space,
        solution_function_space=solution_function_space,
        dirichlet_bc=dirichlet_bc)

Lets try solving this PDE for $m(x)=1$:

In [ ]:
# Create homogeneous parameter m_1(x) = 1
m_1 = dl.Function(parameter_function_space)
m_1.vector().set_local(np.ones(parameter_function_space.dim()))
m_1(.5,.8)

Assemble and solve the PDE

In [ ]:
# Assemble the PDE at m_1
PDE.assemble(m_1)

# Solve the PDE at m_1
u, _ = PDE.solve()

Plot the solution 


In [ ]:
im = dl.plot(u)
plt.colorbar(im)

## 2. Build and solve the Bayesian problem in CUQIpy
  
The goal is to infer the log conductivity profile $m(x)$ given observed data $d$. These observation can be of the potential directly, i.e. $d=u(x)$, or a function of the potential. 

The data $d$ is then given by:

$$ d = \mathcal{G}(m) + \eta$$


where 
- $\eta$ is the measurement noise
- $\mathcal{G}$ is the forward model operator which maps $m$ to the observations.


### 2.1. Create domain geometry 
We model $m$ as a Matern-class random field which lead to the parametrization (Karhunen-Lo\'eve (KL) expansion)
$$
    m(x) = \sum_{i\in \mathbb N} \sqrt{\lambda_i}\theta_i e_i(x)
$$
- $ \lambda_i $ and $ e_i $ are the eigenvalues and eigenvectors of the Matern covariance  operator
- $\theta_i\sim \mathcal N(0,1)$ are i.i.d. random variables.
- Now, $\theta_i$ are the Bayesian parameters

In [ ]:
# Define CUQI geometry on which m is defined
fenics_continuous_geo = cuqipy_fenics.geometry.FEniCSContinuous(parameter_function_space)

# Define the MaternExpansion geometry that maps the i.i.d random variables to Matern field realizations
domain_geometry = cuqipy_fenics.geometry.MaternExpansion(fenics_continuous_geo, length_scale = .1, num_terms=32)

We can look at realizations of Matern class Gaussian random field 

In [ ]:
field_realization = domain_geometry.par2fun(np.random.randn(32))
dl.plot(field_realization)

### 2.2. Create range geometry

In [ ]:
range_geometry= cuqipy_fenics.geometry.FEniCSContinuous(solution_function_space)


### 2.3. Create cuqi forward model

In [ ]:
cuqi_model = cuqi.model.PDEModel(PDE, domain_geometry=domain_geometry,range_geometry=range_geometry)

### 2.4. Create prior

In [ ]:
x = cuqi.distribution.GaussianCov(np.zeros(cuqi_model.domain_dim),1,geometry=domain_geometry)

We can plot prior samples

In [ ]:
prior_samples = x.sample(5)
prior_samples.plot()

### 2.5. Create exact solution and exact data

Create an exact solution:

In [ ]:
np.random.seed(1)
exact_solution =cuqi.samples.CUQIarray(np.random.randn(domain_geometry.par_dim), is_par=True, geometry=domain_geometry )

# plot exact solution
im = exact_solution.plot()
plt.colorbar(im[0])

Create synthesized data that corresponds to the exact_solution

In [ ]:
exact_data = cuqi_model(exact_solution)

# plot exact data
im = range_geometry.plot(exact_data)
plt.colorbar(im[0])

### 2.6. Create likelihood and noisy data
We create the data distribution

In [ ]:
y = cuqi.distribution.GaussianCov(mean=cuqi_model(x), cov=np.ones(cuqi_model.range_dim)*.01**2, geometry=range_geometry)
y

And we create the data

In [ ]:
data = y(x=exact_solution).sample()
data = exact_data

We plot the data

In [ ]:
# plot data
im =range_geometry.plot(data)
plt.colorbar(im[0])

The likelihood distribution:

In [ ]:
y = y(y=data)

### 2.7. Create the posterior

In [ ]:
cuqi_posterior = cuqi.distribution.Posterior(y, x, geometry=domain_geometry)

### 2.8. Sample the posterior
Create the sampler

In [ ]:
Ns = 100
sampler = cuqi.sampler.pCN(cuqi_posterior)

Sample the posterior

In [ ]:
samples = sampler.sample_adapt(Ns,Nb=10)

We plot the samples mean (and the exact solution for reference)

In [ ]:
# plot samples mean
im = samples.plot_mean()
cb = plt.colorbar(im[0])

# plot the exact solution
plt.figure()
im = exact_solution.plot()
cb = plt.colorbar(im[0])
plt.title('Exact solution')

We look at the trace plot

In [ ]:
samples.plot_trace()

We plot the credibility interval

In [ ]:
samples.plot_ci(exact=exact_solution, plot_par=True)
plt.xticks(np.arange(x.dim)[::5],['v'+str(i) for i in range(x.dim)][::5]);

## 3. Use gradient-based sampler

### 3.1. The chain rule

$$\nabla_\theta \mathrm{log}\rho_\mathrm{post}(\theta) \propto \nabla_\theta \mathrm{log}\rho_\mathrm{likelihood}(\mathcal{G}(m(\theta))) + \nabla_\theta \mathrm{log}\rho_\mathrm{prior}(\theta) $$

We have the maps:
- Domain geometry: $z := m(\theta)$
- Model: $y := \mathcal{G}(z) $


By the chain rule we have (for the likelihood part):

$$ \nabla_\theta \mathrm{log}\rho_\mathrm{likelihood}(\mathcal{G}(m(\theta))) = J_{m,\theta}^T(\theta) J_{\mathcal{G}, m}^T(m(\theta)) \nabla_y  \mathrm{log}\rho(\mathcal{G}(m(\theta))) $$ 

- We use adjoint-based method to compute $J_{\mathcal{G}, m}^T(m(\theta)) V$
	- Costs one forward solve and one adjoint solve (cheaper than finite difference approximation)

  

### 3.1 Set the adjoint problem boundary conditions

In [ ]:
adjoint_dirichlet_bc_expr = dl.Constant(0.0)
adjoint_dirichlet_bc = dl.DirichletBC(solution_function_space,
                                      adjoint_dirichlet_bc_expr,
                                      u_boundary) #adjoint problem bcs
PDE.adjoint_dirichlet_bc = adjoint_dirichlet_bc

### 3.2 Check the gradient correctness at a point $x_i$

In [ ]:
# Create x_i
x_i =cuqi.samples.CUQIarray( np.random.randn(domain_geometry.par_dim),is_par=True,geometry= domain_geometry)

Compute the posterior gradient


In [ ]:
print("Posterior gradient (cuqi.model)")
cuqi_grad = cuqi_posterior.gradient(x_i)


Compute the approximate gradient


In [ ]:
print("Scipy approx")
step = 1e-11   # finite diff step
scipy_grad = optimize.approx_fprime(x_i, cuqi_posterior.logpdf, step)

Plot both gradients

In [ ]:
plt.plot(cuqi_grad, label='CUQI')
plt.plot(scipy_grad , '--', label='Approximate')
plt.legend()

### 3.3. Use gradient based sampler (NUTS)

Create the sampler 

In [ ]:
Ns = 100
sampler = cuqi.sampler.NUTS(cuqi_posterior)

Sample using NUTS

In [ ]:
samples = sampler.sample_adapt(Ns,Nb=10)

Plot the mean and the exact solution

In [ ]:
# plot samples mean
im = samples.plot_mean()
cb = plt.colorbar(im[0])

# plot the exact solution
plt.figure()
im = exact_solution.plot()
cb = plt.colorbar(im[0])
plt.title('Exact solution')

Plot trace

In [ ]:
samples.plot_trace()

We plot the credibility interval

In [ ]:
samples.plot_ci(exact=exact_solution, plot_par=True)
plt.xticks(np.arange(x.dim)[::5],['v'+str(i) for i in range(x.dim)][::5]);